In [1]:


import xml.etree.ElementTree as ET
import glob
import os 
import json


def xml_to_yolo_bbox(bbox, w, h):
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


def yolo_to_xml_bbox(bbox, w, h):
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


classes = []
input_dir = "/home/saaho/THYTECH/HelmetDetection/Data/annotations"
output_dir = "labels/"
image_dir = "/home/saaho/THYTECH/HelmetDetection/Data/images"

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)


files = glob.glob(os.path.join(input_dir, '*.xml'))


for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue

    result = []

   
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))


with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))
    
print("XML to YOLO Format Done!!")



BikesHelmets735 image does not exist!
BikesHelmets459 image does not exist!
BikesHelmets35 image does not exist!
XML to YOLO Format Done!!


In [1]:
API_KEY = '0LItzaohvj3LQfnePKBL'

In [4]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="0LItzaohvj3LQfnePKBL"
)

result = CLIENT.infer(your_image.jpg, model_id="hard-hat-workers/13")

NameError: name 'your_image' is not defined

In [3]:
!pip install inference_sdk

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached aiohttp-3.10.11-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached yarl-1.18.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (69 kB)


In [ ]:
import cv2
import os

# Set your image and label paths
image_folder = "/home/saaho/THYTECH/HelmetDetection/HelmetDetection/Data/wokerswithH/worker with helmet images google search"
label_folder = "/home/saaho/THYTECH/HelmetDetection/HelmetDetection/Data/wokerswithH/labels_my-project-name_2025-03-20-10-35-12"


# Get all image files
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Loop through images
for image_file in image_files:
    # Load image
    image_path = os.path.join(image_folder, image_file)
    img = cv2.imread(image_path)
    h, w, _ = img.shape
    
    # Find corresponding label file
    label_file = os.path.join(label_folder, image_file.replace(".jpg", ".txt").replace(".png", ".txt"))
    
    if os.path.exists(label_file):
        with open(label_file, "r") as file:
            lines = file.readlines()
            for line in lines:
                data = line.strip().split()
                class_id = int(data[0])
                x_center, y_center, bbox_w, bbox_h = map(float, data[1:])

                # Convert YOLO format to pixel coordinates
                x1 = int((x_center - bbox_w / 2) * w)
                y1 = int((y_center - bbox_h / 2) * h)
                x2 = int((x_center + bbox_w / 2) * w)
                y2 = int((y_center + bbox_h / 2) * h)

                # Draw bounding box
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, f"Class {class_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the image
    cv2.imshow("Image with Labels", img)
    
    # Wait for key press before showing the next image
    key = cv2.waitKey(0)  
    if key == 27:  # Press 'ESC' to exit early
        break

cv2.destroyAllWindows()


In [3]:
import os 

label_folder = "/home/saaho/THYTECH/HelmetDetection/HelmetDetection/Data/wokerswithH/labels_my-project-name_2025-03-20-10-35-12"

label_txt = os.listdir(label_folder)
for file in label_txt:
    file_path=  os.path.join(label_folder , file)
    with open(file_path , 'r') as f:
        cont = f.readlines()
        for d in cont:
            print(file , 'Content:' ,d)

16684989-beaming-construction-worker-wearing-yellow-safety-helmet-cheerful-portrait.txt Content: 0 0.676297 0.234936 0.222877 0.469871
The-Types-Of-Safety-Helmets-For-Head-Protection.txt Content: 0 0.473467 0.350680 0.182783 0.254463

The-Types-Of-Safety-Helmets-For-Head-Protection.txt Content: 0 0.650943 0.345378 0.162736 0.289805
head-protection-thumb-1038x576-1.txt Content: 0 0.715802 0.333410 0.396226 0.666820
roof-headphones-yellow-helmet-working-iron-cutting-machine-factory-213418200.txt Content: 0 0.371462 0.244718 0.327830 0.445198
image.txt Content: 0 0.559649 0.297368 0.466667 0.568421
and-plaid-shirt-diverse-industrial-employment-professional-stockgraphy-photo.txt Content: 0 0.586675 0.241055 0.244104 0.482110
blog-banner-safety-helmets.txt Content: 0 0.441627 0.260318 0.321934 0.498821
construction-worker-planning-stockcake.txt Content: 0 0.493860 0.310526 0.447368 0.492982
1200px-Cologne_Germany_Industrial-work-with-Personal-Protective-Equipment-04.txt Content: 0 0.525088 

In [1]:
import os
import shutil

# Paths
label_folder = "/home/saaho/THYTECH/HelmetDetection/HelmetDetection/Data/wokerswithH/labels_my-project-name_2025-03-20-10-35-12"  # Replace with your label folder
fixed_label_folder = "/home/saaho/THYTECH/HelmetDetection/HelmetDetection/Data/wokerswithH/correctelabel" # Folder for corrected labels
os.makedirs(fixed_label_folder, exist_ok=True)

# Process each label file
for label_file in os.listdir(label_folder):
    if label_file.endswith(".txt"):
        label_path = os.path.join(label_folder, label_file)
        
        with open(label_path, "r") as file:
            lines = file.readlines()
        
        updated_lines = []
        modified = False
        
        for line in lines:
            data = line.strip().split()
            class_id = data[0]
            
            if class_id == "0":  # Change Class 0 to Class 1
                data[0] = "1"
                modified = True
            
            updated_lines.append(" ".join(data))

        if modified:
            new_label_path = os.path.join(fixed_label_folder, label_file)
            with open(new_label_path, "w") as file:
                file.write("\n".join(updated_lines))
            
            print(f"Fixed: {label_file} (Class 0 → Class 1)")

print("✅ All mislabeled files have been corrected and saved in:", fixed_label_folder)


Fixed: 16684989-beaming-construction-worker-wearing-yellow-safety-helmet-cheerful-portrait.txt (Class 0 → Class 1)
Fixed: The-Types-Of-Safety-Helmets-For-Head-Protection.txt (Class 0 → Class 1)
Fixed: head-protection-thumb-1038x576-1.txt (Class 0 → Class 1)
Fixed: roof-headphones-yellow-helmet-working-iron-cutting-machine-factory-213418200.txt (Class 0 → Class 1)
Fixed: image.txt (Class 0 → Class 1)
Fixed: and-plaid-shirt-diverse-industrial-employment-professional-stockgraphy-photo.txt (Class 0 → Class 1)
Fixed: blog-banner-safety-helmets.txt (Class 0 → Class 1)
Fixed: construction-worker-planning-stockcake.txt (Class 0 → Class 1)
Fixed: images86.txt (Class 0 → Class 1)
Fixed: image16.txt (Class 0 → Class 1)
Fixed: image18.txt (Class 0 → Class 1)
Fixed: 94141464-young-worker-man-with-safety-helmet-in-construction.txt (Class 0 → Class 1)
Fixed: Industrial-Construction-Hat-Outdoor-ABS-Safety-Helmet-for-Worker-Safety.txt (Class 0 → Class 1)
Fixed: mockup-black-male-construction-worker-wit